In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


leakage_cols = ["Fire_Risk","Fire_Risk_Label","Alert_Triggered","Fire_Sensor","Sprinkler_Status"]
X = df.drop(columns=[c for c in leakage_cols if c in df.columns])
y = df["Fire_Risk"]




In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

In [ ]:
cat_cols = X.select_dtypes(include="object").columns
num_cols = X.select_dtypes(exclude="object").columns

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ]
)


In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline

# KNN pipeline
knn_pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", KNeighborsClassifier(n_neighbors=25, weights="distance"))
])


knn_pipe.fit(X_train, y_train)


def best_threshold_for_f1(y_true, probas, t_min=0.2, t_max=0.9, step=0.01):
    best_t, best_f1 = 0.5, -1
    for t in np.arange(t_min, t_max + step, step):
        y_pred = (probas >= t).astype(int)
        f1 = f1_score(y_true, y_pred)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    return best_t, best_f1

# -----------------------------
# TRAIN ACCURACY
# -----------------------------
train_probas = knn_pipe.predict_proba(X_train)[:, 1]
t_best_train, f1_best_train = best_threshold_for_f1(y_train.values, train_probas)
y_train_pred = (train_probas >= t_best_train).astype(int)

print("KNN Train Accuracy:", accuracy_score(y_train, y_train_pred))
print("KNN Train Best Threshold:", round(t_best_train,2), "| Best F1:", round(f1_best_train,4))
print("Train Confusion Matrix:\n", confusion_matrix(y_train, y_train_pred))
print("Train Classification Report:\n", classification_report(y_train, y_train_pred))

# -----------------------------
# TEST ACCURACY
# -----------------------------
test_probas = knn_pipe.predict_proba(X_test)[:, 1]
t_best, f1_best = best_threshold_for_f1(y_test.values, test_probas)
y_test_pred = (test_probas >= t_best).astype(int)

print("\n KNN Test Threshold:", round(t_best,2), "| Best F1:", round(f1_best,4))
print(" KNN Test Accuracy:", accuracy_score(y_test, y_test_pred))
print("Test Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))
print("Test Classification Report:\n", classification_report(y_test, y_test_pred))



ModuleNotFoundError: No module named 'sklearn'